In [21]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!pip install datasets

In [104]:
from datasets import Dataset,load_dataset
import pandas as pd

# Load your data
df = pd.read_csv("./model_data_2.csv",encoding='utf-8')

df.dropna(inplace=True)

# Fake #Reliable #Unreliable  #
df['label']=df['label'].map({'SUPPORTS':'Reliable','false':'Fake','REFUTES':'Fake','mixture':'Unreliable',
                             'Not Sure':'Unreliable','unknown':'NOT ENOUGH INFO','mostly-false':'Unreliable',
                             'mostly-true':'Reliable','Realiable':'Reliable','true':'Reliable'}).fillna(df['label'])

df['label']=df['label'].map({'Unreliable':0,'Fake':1,'Reliable':2,'NOT ENOUGH INFO':3})
df['label'].value_counts()

# dataset = load_dataset('csv', data_files='model_data_2.csv')


,count
label,
1,3259
2,2569
0,1791
3,816


In [105]:
df['label'].isna().sum() #should be 0

np.int64(0)

In [100]:
# df['label'].unique()

array(['Fake', 'Not Sure', 'SUPPORTS', 'REFUTES', 'NOT ENOUGH INFO',
       'Realiable', 'Unreliable', 'true', 'false', 'mixture',
       'mostly-true', 'mostly-false', 'unknown'], dtype=object)

In [40]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"  # You can swap with other LLMs
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [106]:

# Step 3: Define the tokenization function
def tokenize_fn(example):
    return tokenizer(example['cleaned_text'],  # Replace with your actual text column
                     truncation=True,
                     padding='max_length',
                     max_length=512)

dataset=Dataset.from_pandas(df[['cleaned_text','label']])
# Step 4: Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_fn)

# Optional: Check a sample
print(tokenized_dataset[0])


Map:   0%|          | 0/8435 [00:00<?, ? examples/s]

{'cleaned_text': 'tiny implantable device shortcircuits hunger pangs aids weight lossmadison wis  more than   million adults and children worldwide are obese according to a   study that called the growing number and weightrelated health problems a rising pandemic  new batteryfree easily implantable weightloss devices developed by engineers at the university of wisconsinmadison could offer a promising new weapon for battling the bulge  in laboratory testing the devices helped rats shed almost   percent of their body weight results of the study were published today dec     in the journal nature communications  measuring less than   centimeter across or about a third of the area of a us penny the tiny devices  which are safe for use in the body and implantable via a minimally invasive procedure  generate gentle electric pulses from the stomachs natural churning motions and deliver them to the vagus nerve which links the brain and the stomach  that gentle stimulation dupes the brain into t

In [107]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split


checkpoint='bert-base-uncased'

# Split dataset
train_test = tokenized_dataset.train_test_split(test_size=0.2)
train_data = train_test['train']
test_data = train_test['test']

# Load model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['label'].unique()))
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [108]:
# df['label'].value_counts()

In [109]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [110]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./bert_misinformation",
    # evaluation_strategy="epoch",
    report_to="none",
    logging_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
)

# Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
)

trainer.train()

<ipython-input-110-86eee8f458e1>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


KeyboardInterrupt: 

In [19]:
# def tokenize_fn(example):
#     # Convert 'cleaned_text' to a string before tokenization
#     example['cleaned_text'] = str(example['cleaned_text'])
#     return tokenizer(example["cleaned_text"], truncation=True, padding="max_length", max_length=512)

# # Recreate the dataset to reset the internal state
# dataset = Dataset.from_pandas(df[['cleaned_text', 'label']])

# tokenized_dataset = dataset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/7626 [00:00<?, ? examples/s]

ArrowInvalid: Column 1 named label expected length 4393979 but got length 1000